In [1]:
from fbprophet import Prophet
from prophet.diagnostics import cross_validation
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
df = pd.read_pickle('../Data/ELC_12-14_15min_dropped.pkl')
df

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_360,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369
2012-01-01 00:15:00,3.807107,22.759602,77.324066,136.178862,70.731707,351.190476,9.609949,279.461279,75.174825,87.096774,...,223.684211,128.479657,28500.0,1729.957806,1704.545455,15.645372,12.873025,504.828797,63.439065,761.730205
2012-01-01 00:30:00,5.076142,22.759602,77.324066,136.178862,73.170732,354.166667,9.044658,279.461279,73.426573,84.946237,...,236.842105,127.765882,26400.0,1654.008439,1659.090909,15.645372,13.458163,525.021949,60.100167,702.346041
2012-01-01 00:45:00,3.807107,22.759602,77.324066,140.243902,69.512195,348.214286,8.479367,279.461279,75.174825,91.397849,...,214.912281,114.204140,25200.0,1333.333333,1636.363636,15.645372,10.532475,526.777875,56.761269,696.480938
2012-01-01 01:00:00,3.807107,22.759602,77.324066,140.243902,75.609756,339.285714,7.348785,279.461279,68.181818,88.172043,...,200.657895,112.062812,23800.0,1324.894515,1636.363636,15.645372,14.628438,539.947322,63.439065,693.548387
2012-01-01 01:15:00,5.076142,22.048364,77.324066,146.341463,73.170732,342.261905,6.783493,265.993266,69.930070,86.021505,...,199.561404,112.776588,23700.0,1118.143460,1659.090909,15.645372,9.947338,556.628622,68.447412,723.607038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-31 23:00:00,2.538071,22.048364,1.737619,150.406504,85.365854,303.571429,11.305822,282.828283,68.181818,72.043011,...,206.140351,276.945039,28200.0,1616.033755,1363.636364,29.986962,5.851375,697.102722,176.961603,651.026393
2014-12-31 23:15:00,2.538071,21.337127,1.737619,166.666667,81.707317,324.404762,11.305822,252.525253,64.685315,72.043011,...,192.982456,279.800143,28300.0,1569.620253,1340.909091,29.986962,9.947338,671.641791,168.614357,669.354839
2014-12-31 23:30:00,2.538071,20.625889,1.737619,162.601626,82.926829,318.452381,10.175240,242.424242,61.188811,74.193548,...,191.885965,284.796574,27800.0,1556.962025,1318.181818,27.379400,9.362200,670.763828,153.589316,670.087977
2014-12-31 23:45:00,1.269036,21.337127,1.737619,166.666667,85.365854,285.714286,10.175240,225.589226,64.685315,72.043011,...,190.789474,246.252677,28000.0,1443.037975,909.090909,26.075619,4.095963,664.618086,146.911519,646.627566


In [ ]:
df = df.resample('1D').mean()
df

In [ ]:
df = df.loc['2012-01-01':'2014-12-31']
df

In [ ]:
df_marked = df.copy()

In [ ]:
#starting = []
#for column in tqdm(range(len(df.columns))):
#    for i in range(len(df_marked)-1):
#        if df.iloc[i,column] == 0 and df.iloc[i+1,column] == 0:
#            df_marked.iloc[i,column] = np.nan
#        elif df.iloc[i,column] == 0 and df.iloc[i+1,column] > 0:
#            df_marked.iloc[i,column] = np.nan
#            starting.append(i)
#            break
#        
#df_marked
        

In [7]:
df_avg_1h = []
for i in tqdm(range(df_marked.shape[0])):
    df_avg_1h.append(df_marked.iloc[i,:].sum()/len(df_marked.iloc[i,:].dropna()))
df_avg_1h = pd.DataFrame(df_avg_1h,index=df_marked.index,columns=['avg_1h'])
df_avg_1h

100%|██████████| 1096/1096 [00:00<00:00, 9898.74it/s]


,avg_1h
2012-01-01,399.521686
2012-01-02,590.506390
2012-01-03,591.221033
2012-01-04,593.381152
2012-01-05,595.020094
...,...
2014-12-27,546.376092
2014-12-28,540.622141
2014-12-29,542.862690
2014-12-30,541.107932


In [9]:
df_avg_1h.sort_values(by='avg_1h')

,avg_1h
2014-12-25,309.387136
2013-12-25,356.607385
2014-01-01,366.212499
2013-01-01,373.608961
2012-12-25,386.249166
...,...
2012-08-20,905.769602
2012-07-18,911.334779
2012-07-17,912.776600
2012-08-10,931.575673


In [ ]:
plt.figure(figsize=(20,10))
plt.plot(df_avg_1h)

In [ ]:
#df_avg_1h.to_pickle('../Data/df_avg_1h.pkl')

In [ ]:
df_avg_1h['ds'] = df_avg_1h.index
df_avg_1h['y'] = df_avg_1h['avg_1h']
df_avg_1h = df_avg_1h.drop('avg_1h',axis=1)

In [ ]:
df_avg_1h

In [ ]:
training_x = df_avg_1h.iloc[:int(len(df_avg_1h)*0.8)]
val_x = df_avg_1h.iloc[int(len(df_avg_1h)*0.8):int(len(df_avg_1h)*0.9)]
testing_x = df_avg_1h.iloc[int(len(df_avg_1h)*0.9):]

In [ ]:
print(training_x.shape)
print(val_x.shape)
print(testing_x.shape)


In [ ]:
m = Prophet(growth='linear',yearly_seasonality=True,weekly_seasonality=True,daily_seasonality=True,seasonality_mode='multiplicative')
m.add_country_holidays(country_name='PT')
m.fit(training_x)

In [ ]:
future = val_x[['ds']]
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(val_x['y'],label='Actual')
plt.plot(forecast['ds'],forecast['yhat'],label='Predicted')

In [ ]:
print(mean_absolute_percentage_error(val_x['y'],forecast['yhat']))

In [ ]:
future = testing_x[['ds']]
forecast = m.predict(future)
plt.figure(figsize=(20,10))
plt.plot(testing_x['y'],label='Actual')
plt.plot(forecast['ds'],forecast['yhat'],label='Predicted')

In [ ]:
print(mean_absolute_percentage_error(testing_x['y'],forecast['yhat']))

In [ ]:
fig = m.plot_components(forecast)

# Hyperparameter tuning with Bayes Optimization

In [ ]:
from bayes_opt import BayesianOptimization

In [ ]:
def prophet_objective(changepoint_prior_scale, seasonality_prior_scale, holiday_prior_scale, interval_width, changepoint_range):
    model = Prophet(
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        holidays_prior_scale= holiday_prior_scale,
        interval_width= interval_width,
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        seasonality_mode='multiplicative',
        changepoint_range=changepoint_range
    )
    model.add_country_holidays(country_name='PT')
    model.fit(training_x)

    forecast = model.predict(val_x[['ds']])
    mape = mean_absolute_percentage_error(val_x['y'],forecast['yhat'])
    return -mape

# Set bounds for the hyperparameters
bounds = {
    'changepoint_prior_scale': (0.001, 0.5),
    'seasonality_prior_scale': (0.01, 10),
    'holiday_prior_scale': (0.01, 10),
    'interval_width': (0.01,0.95),
    'changepoint_range': (0.8,0.95)
}

# Initialize the Bayesian optimization
optimizer = BayesianOptimization(
    f=prophet_objective,
    pbounds=bounds,
    random_state=42,
)

# Perform the optimization
optimizer.maximize(init_points=10, n_iter=20)

In [ ]:
# Extract the best hyperparameters
best_params = optimizer.max['params']

# Train and evaluate the model with the best hyperparameters
best_model = Prophet(
    changepoint_prior_scale=best_params['changepoint_prior_scale'],
    seasonality_prior_scale=best_params['seasonality_prior_scale']
)
best_model.fit(pd.concat([training_x, val_x]))
best_forecast = best_model.predict(testing_x[['ds']])
best_mape = mean_absolute_percentage_error(testing_x['y'], best_forecast['yhat'])

print(f"Best hyperparameters: {best_params}")
print(f"Best mape: {best_mape}")

In [ ]:
future = testing_x[['ds']]
forecast = m.predict(future)
plt.figure(figsize=(20,10))
plt.plot(testing_x['y'],label='Actual')
plt.plot(forecast['ds'],forecast['yhat'],label='Predicted')